In [6]:
import sys
sys.path.insert(0, '../')
%reload_ext autoreload
%autoreload 2
import logging
import torch
from nowcasting.hko.dataloader import HKOIterator
from nowcasting.config import cfg
import numpy as np

In [2]:
IN_LEN = cfg.HKO.BENCHMARK.IN_LEN
OUT_LEN = cfg.HKO.BENCHMARK.OUT_LEN

In [3]:
train_hko_iter = HKOIterator(pd_path=cfg.HKO_PD.RAINY_TRAIN,
                                 sample_mode="random",
                                 seq_len=IN_LEN+OUT_LEN)

valid_hko_iter = HKOIterator(pd_path=cfg.HKO_PD.RAINY_VALID,
                                 sample_mode="sequent",
                                 seq_len=IN_LEN+OUT_LEN,
                                 stride=cfg.HKO.BENCHMARK.STRIDE)

In [4]:
train_batch_size = 2
test_batch_size = 2
max_iterations = 2000
test_and_save_checkpoint_iterations = 100

In [7]:
from nowcasting.models.forecaster import Forecaster
from nowcasting.models.encoder import Encoder
from nowcasting.models.convLSTM import ConvLSTM
from collections import OrderedDict

In [8]:
# build model
encoder_params = [
    [
        OrderedDict({'conv1_leaky_1': [1, 8, 7, 5, 1]}),
        OrderedDict({'conv2_leaky_1': [64, 192, 5, 3, 1]}),
        OrderedDict({'conv3_leaky_1': [192, 192, 3, 2, 1]}),
    ],

    [
        ConvLSTM(8, 64, 3),
        ConvLSTM(192, 192, 3),
        ConvLSTM(192, 192, 3)
    ]
]

In [9]:
encoder = Encoder(encoder_params[0], encoder_params[1]).to(cfg.GLOBAL.DEVICE)
encoder

Encoder(
  (stage1): Sequential(
    (conv1_leaky_1): Conv2d(1, 8, kernel_size=(7, 7), stride=(5, 5), padding=(1, 1))
    (leaky_conv1_leaky_1): LeakyReLU(negative_slope=0.2, inplace)
  )
  (rnn1): ConvLSTM(
    (_cell): ConvLSTMCell(
      (_conv): Conv2d(72, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (stage2): Sequential(
    (conv2_leaky_1): Conv2d(64, 192, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1))
    (leaky_conv2_leaky_1): LeakyReLU(negative_slope=0.2, inplace)
  )
  (rnn2): ConvLSTM(
    (_cell): ConvLSTMCell(
      (_conv): Conv2d(384, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (stage3): Sequential(
    (conv3_leaky_1): Conv2d(192, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (leaky_conv3_leaky_1): LeakyReLU(negative_slope=0.2, inplace)
  )
  (rnn3): ConvLSTM(
    (_cell): ConvLSTMCell(
      (_conv): Conv2d(384, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
)

In [10]:
forecaster_params = [
    [
        OrderedDict({'deconv1_leaky_1': [192, 192, 4, 2, 1]}),
        OrderedDict({'deconv2_leaky_1': [192, 64, 5, 3, 1]}),
        OrderedDict({
            'deconv3_leaky_1': [64, 8, 7, 5, 1],
            'conv3_leaky_2': [8, 8, 3, 1, 1],
            'conv3_3': [8, 1, 1, 1, 0] # 忘了删除激活函数了，妈的
            # 忘了卷积层，分类
        }),
    ],

    [
        ConvLSTM(192, 192, 3),
        ConvLSTM(192, 192, 3),
        ConvLSTM(64, 64, 3)
    ]
]

In [11]:
forecaster = Forecaster(forecaster_params[0], forecaster_params[1]).to(cfg.GLOBAL.DEVICE)
forecaster

Forecaster(
  (rnn3): ConvLSTM(
    (_cell): ConvLSTMCell(
      (_conv): Conv2d(384, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (stage3): Sequential(
    (deconv1_leaky_1): ConvTranspose2d(192, 192, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (leaky_deconv1_leaky_1): LeakyReLU(negative_slope=0.2, inplace)
  )
  (rnn2): ConvLSTM(
    (_cell): ConvLSTMCell(
      (_conv): Conv2d(384, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (stage2): Sequential(
    (deconv2_leaky_1): ConvTranspose2d(192, 64, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1))
    (leaky_deconv2_leaky_1): LeakyReLU(negative_slope=0.2, inplace)
  )
  (rnn1): ConvLSTM(
    (_cell): ConvLSTMCell(
      (_conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (stage1): Sequential(
    (deconv3_leaky_1): ConvTranspose2d(64, 8, kernel_size=(7, 7), stride=(5, 5), padding=(1, 1))
    (leaky_deconv3_leaky_1): LeakyReLU(negative_slope=0.

In [10]:
from nowcasting.models.model import EF

In [11]:
encoder_forecaster = EF(encoder, forecaster).to(cfg.GLOBAL.DEVICE)

In [12]:
encoder_forecaster

EF(
  (encoder): Encoder(
    (stage1): Sequential(
      (conv1_1): Conv2d(1, 8, kernel_size=(7, 7), stride=(5, 5), padding=(1, 1))
      (relu_conv1_1): ReLU(inplace)
    )
    (rnn1): ConvLSTM(
      (_cell): ConvLSTMCell(
        (_conv): Conv2d(72, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (stage2): Sequential(
      (conv2_1): Conv2d(64, 192, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1))
      (relu_conv2_1): ReLU(inplace)
    )
    (rnn2): ConvLSTM(
      (_cell): ConvLSTMCell(
        (_conv): Conv2d(384, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (stage3): Sequential(
      (conv3_1): Conv2d(192, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (relu_conv3_1): ReLU(inplace)
    )
    (rnn3): ConvLSTM(
      (_cell): ConvLSTMCell(
        (_conv): Conv2d(384, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
  )
  (forecaster): Forecaster(
    (rnn3): ConvLSTM(
      (_cell): Co

### 训练

#### 参数

In [13]:
from torchvision import transforms
from torch.optim import lr_scheduler

In [14]:
LR = 1e-4
optimizer = torch.optim.Adam(encoder_forecaster.parameters(), lr=LR)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.1)
criterion = torch.nn.MSELoss(reduction='mean')
test_iteration_interval = 100

In [15]:
from nowcasting.hko.evaluation import HKOEvaluation
evaluater = HKOEvaluation(seq_len=OUT_LEN, use_central=False)

In [16]:
train_loss = 0.0
test_loss = 0.0

for itera in range(max_iterations):
    if itera%test_iteration_interval==0:
        train_loss = 0.0
        test_loss = 0.0
        evaluater.clear_all()
        with torch.no_grad():
            pass
    train_batch, train_mask, sample_datetimes, _ = \
                train_hko_iter.sample(batch_size=train_batch_size)
    train_batch = torch.from_numpy(train_batch.astype(np.float32)).to(cfg.GLOBAL.DEVICE)/255.0
    train_data = train_batch[:IN_LEN, ...]
    train_label = train_batch[IN_LEN:IN_LEN+OUT_LEN, ...]
    mask = torch.from_numpy(train_mask[IN_LEN:IN_LEN+OUT_LEN, ...].astype(int)).to(cfg.GLOBAL.DEVICE)
    
    encoder_forecaster.train()
    optimizer.zero_grad()
    output = encoder_forecaster(train_data)
    loss = criterion(output, train_label)
    loss.backward()
    optimizer.step()
#     train_loss += loss.item()
    train_loss = loss.item()
    evaluater.update(train_label.cpu().numpy(), output.detach().cpu().numpy(), mask.cpu().numpy())
    print(train_loss)

0.7263621687889099
0.821571409702301
0.7881657481193542
0.7651374936103821
0.6349201202392578
0.7382422089576721
0.6729480028152466
0.6955388188362122
0.6149008870124817
0.6411648988723755
0.6127393841743469
0.5829384326934814
0.5575474500656128
0.5201427936553955
0.5508731007575989
0.46293848752975464
0.49638161063194275
0.4191792607307434
0.46300461888313293
0.4250716269016266
0.4182124137878418
0.4105169475078583
0.4006730914115906
0.3718911111354828
0.3777596056461334
0.35242393612861633
0.33277982473373413
0.33807477355003357
0.29463711380958557
0.3016614615917206
0.2887493669986725
0.2396143078804016
0.24983742833137512
0.22717474400997162
0.22936341166496277
0.22979500889778137
0.22087819874286652
0.20947781205177307
0.190176859498024
0.17819832265377045
0.17923912405967712
0.1791492998600006
0.1562056541442871
0.15579786896705627
0.1500856727361679
0.1437840610742569
0.1425778865814209
0.13841843605041504
0.1223856583237648
0.12771092355251312
0.12090755254030228
0.121836423873

KeyboardInterrupt: 